# Popis systému

Regulátor může být popsán například pomocí následjícího obrázku

<img src="Adaptive/Fig_feedb_controller.png" width=700>

Výstup systému počítáme takto
$$  y(k+h)= w \cdot x
=\begin{bmatrix}
w_0 & w_1 & \dots & w_{n+m} 
\end{bmatrix}
\cdot
\begin{bmatrix}
1\\  y(k)\\ y(k-1)\\ \vdots \\ y(k-n+1)\\ u(k)\\ u(k-1)\\ \vdots \\ u(k-m+1)
\end{bmatrix},
$$


Definujeme si chybu
$$ \normalsize{e(k+1)=y(k+1)-\tilde{y}(k+1),}$$
  

Chybu se budeme snažit minimalizovat lépe zvolenými vahami

$$\textbf{w}=\textbf{w}+\small{\Delta}\textbf{w}.$$

Derivujeme chybu podle váhy a vzniknuvší gradient přidáme k aktuálním vahám

$$\Delta\textbf{w}(k)=
-\dfrac{\mu_w}{2}\dfrac{\partial e^2(k)}{\partial \textbf{w}}
=\mu_w\cdot e(k) \cdot \textbf{x}^T(k-1) \ , $$


Definice vstupu a převod na jiné popisy

In [7]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from math import e
import sympy as sp
from mpmath import e
import ipywidgets
from IPython.display import display, Math, Latex

dt = .05  #[sec]
t = np.arange(0, 10, dt)
N = len(t)  # delka dat

u = np.sin(2 * np.pi / 10 * t)
u = np.sign(u)


from scipy.integrate import odeint
import matplotlib.pyplot as plt

def fdxdt(x, t, u, Omega, eta, b0,
          b1):  # x=[x1 x2 ... xn] vektor hodnot n stavovych velicin
    dx1dt = -Omega0**2 * x[1] - b0 * u
    dx2dt = -2 * eta * Omega0 * x[1] - b1 * u + x[0]
    return (dx1dt, dx2dt)

Omega0 = 10
eta = .1
b0 = Omega0**2
b1 = 0

y_ref = np.zeros(N)
x10 = 0
x20 = 0  # poc. podm

x0 = [x10, x20]

for i in range(0, N - 1):
    tt = [t[i], t[i + 1]]  # [t1 t2]
    x = odeint(fdxdt, x0, t,
               (u[i], Omega0, eta, b0,
                b1))  #returns x=[ [x1(t1) x2(t1)] [x1(t2) x2(t2)]]
    #    x=odeint(fdxdt,x0,tt,args=(u[i],)) # <-- pokud je jen jeden extra argument, musi se tak
    y_ref[i + 1] = -x[1, 1]
    x0 = x[1, :]  # jako nove poc. podm pro dalsi integraci

fig = plt.figure()
fig.canvas.layout.width = '800px'
plt.plot(y_ref)

# Create plot, that will be updated
fig = plt.figure()
fig.canvas.layout.width = '800px'
ax = fig.add_subplot(1, 1, 1)
y = y_ref  # Empty plot container
y_ref_pl, = ax.plot(t, y_ref, label="Skutečnost")
y_pl, = ax.plot(t, y, label="Identifikovaná")
plt.legend(loc="upper right")

# Create widgets - Box with sliders
mu_sel = [0.001, 0.01, 0.1, 1, 10]
dt_sel = [0.001, 0.01, 0.1, 1]

layout = ipywidgets.Layout( display='flex', justify_content='space-around', width='1000px')

dt_slid = ipywidgets.SelectionSlider(options=[("%g"%i,i) for i in dt_sel], value=0.01, description='Sampling')
mu_slid = ipywidgets.SelectionSlider(options=[("%g"%i,i) for i in mu_sel], description='Learning rate')
epochs_slid = ipywidgets.IntSlider(min=1, max=20, step=1, value=1, description='Epochs')
random_slid = ipywidgets.RadioButtons( options=[0, 1], description='Random initial weights' )
normalize_slid = ipywidgets.RadioButtons(options=[0, 1], description='Normalize mu')

vbox = ipywidgets.VBox([dt_slid, mu_slid, epochs_slid])
vbox2 = ipywidgets.VBox([random_slid, normalize_slid])
hbox = ipywidgets.HBox([vbox, vbox2], layout=layout)


def update(mu, epochs, random=0, minorm=0):
    
    y = np.zeros(len(y_ref))
    if random:
        w = np.random.randn(2)/3
    else:
        w = np.zeros(2)
    X = np.zeros(2)
    X[0] = 1
    y = np.zeros(len(y_ref))
    e = np.zeros(len(y_ref))

    w_all = np.zeros((len(y_ref), 2))
    y[0]=y_ref[0]
    
    for ep in range(epochs):
        for i in range(1, len(y_ref)):
            X[1] = y_ref[i - 1]
            y[i] = np.dot(w, X)
            e[i]= y_ref[i] - y[i]
            #minorm = mu/(5+np.dot(X,X.T))
            dw = mu/(1+sum(X*X)) * e[i] * X
            w = w + dw
            if ep == epochs - 1:
                w_all[i] = w 
                
    ### Update plot ###
    y_pl.set_ydata(y)
    y_ref_pl.set_ydata(y_ref)
    
    fig.canvas.draw_idle()

    
    print('\n\nAbsolutní chyba za poslední epochu',sum(abs(e)))
    print(f'\nIdentifikované parametry: c = {w[0]}   a = {w[1]}')

    display(Math(r'y(k)=a y(k-1) + c'))
    display(Math(fr'y(k)={w[1]} \cdot y(k-1) + {w[0]}'))
    
interactive_plot = ipywidgets.interactive_output(update, {"mu": mu_slid, "epochs": epochs_slid,
                                                          "random": random_slid, "minorm": normalize_slid})

display(hbox, interactive_plot)









'''



plt.figure(figsize=(12,7))
plt.subplot(4, 1, 1)
plt.plot(t, y); plt.grid(); plt.xlabel('t')
plt.ylabel("u4 predikované")

plt.subplot(4, 1, 2)
plt.plot(t, y_ref); plt.grid(); plt.xlabel('t')
plt.ylabel("u4 skutečné")

plt.subplot(4, 1, 3)
plt.plot(t, e); plt.grid(); plt.xlabel('t')
plt.ylabel("Chyba")

plt.subplot(4, 1, 4)
plt.plot(t, wall); plt.grid(); plt.xlabel('t')
plt.ylabel("Hodnoty vah")

plt.suptitle("Predikovaná vs. skutečná hodnota, chyba a váhy", fontsize=20)
plt.subplots_adjust(top=0.88)
plt.show()

print(sum(abs(e)))

print('parametry systému [] ', w)
'''

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

'\n\n\n\nplt.figure(figsize=(12,7))\nplt.subplot(4, 1, 1)\nplt.plot(t, y); plt.grid(); plt.xlabel(\'t\')\nplt.ylabel("u4 predikované")\n\nplt.subplot(4, 1, 2)\nplt.plot(t, y_ref); plt.grid(); plt.xlabel(\'t\')\nplt.ylabel("u4 skutečné")\n\nplt.subplot(4, 1, 3)\nplt.plot(t, e); plt.grid(); plt.xlabel(\'t\')\nplt.ylabel("Chyba")\n\nplt.subplot(4, 1, 4)\nplt.plot(t, wall); plt.grid(); plt.xlabel(\'t\')\nplt.ylabel("Hodnoty vah")\n\nplt.suptitle("Predikovaná vs. skutečná hodnota, chyba a váhy", fontsize=20)\nplt.subplots_adjust(top=0.88)\nplt.show()\n\nprint(sum(abs(e)))\n\nprint(\'parametry systému [] \', w)\n'

In [7]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from math import e
import sympy as sp
from mpmath import e
import ipywidgets
from IPython.display import display, Math, Latex

dt = .05  #[sec]
t = np.arange(0, 10, dt)
N = len(t)  # delka dat

u = np.sin(2 * np.pi / 10 * t)
u = np.sign(u)


from scipy.integrate import odeint
import matplotlib.pyplot as plt

def fdxdt(x, t, u, Omega, eta, b0,
          b1):  # x=[x1 x2 ... xn] vektor hodnot n stavovych velicin
    dx1dt = -Omega0**2 * x[1] - b0 * u
    dx2dt = -2 * eta * Omega0 * x[1] - b1 * u + x[0]
    return (dx1dt, dx2dt)

Omega0 = 10
eta = .1
b0 = Omega0**2
b1 = 0

y_ref = np.zeros(N)
x10 = 0
x20 = 0  # poc. podm

x0 = [x10, x20]

for i in range(0, N - 1):
    tt = [t[i], t[i + 1]]  # [t1 t2]
    x = odeint(fdxdt, x0, t,
               (u[i], Omega0, eta, b0,
                b1))  #returns x=[ [x1(t1) x2(t1)] [x1(t2) x2(t2)]]
    #    x=odeint(fdxdt,x0,tt,args=(u[i],)) # <-- pokud je jen jeden extra argument, musi se tak
    y_ref[i + 1] = -x[1, 1]
    x0 = x[1, :]  # jako nove poc. podm pro dalsi integraci

# Create plot, that will be updated
fig = plt.figure()
fig.canvas.layout.width = '800px'

y = e = np.zeros(N)  # Empty plot containers

ax = plt.subplot(3, 1, 1)
y_ref_pl, = ax.plot(t, y_ref, label="Skutečnost"); plt.grid(); plt.xlabel('t')
y_pl, = ax.plot(t, y, label="Identifikovaná")
plt.legend(loc="upper right")

ax2 = plt.subplot(3, 1, 2)

plt.ylabel("Chyba")

ax3 = plt.subplot(3, 1, 3)
plt.ylabel("Váhy")

plt.suptitle("Predikovaná vs. skutečná hodnota, chyba a váhy", fontsize=20)
plt.subplots_adjust(top=0.88)
plt.show()

# Create widgets - Box with sliders
mu_sel = [0.001, 0.01, 0.1, 1, 10]
dt_sel = [0.001, 0.01, 0.1, 1]

layout = ipywidgets.Layout( display='flex', justify_content='space-around', width='1000px')

mu_slid = ipywidgets.SelectionSlider(options=[("%g"%i,i) for i in mu_sel], description='Learning rate')
epochs_slid = ipywidgets.IntSlider(min=1, max=20, step=1, value=1, description='Epochs')
random_slid = ipywidgets.RadioButtons( options=[0, 1], description='Random w' )
normalize_slid = ipywidgets.RadioButtons(options=[0, 1], description='Normalize mu')

vbox = ipywidgets.VBox([mu_slid, epochs_slid])
vbox2 = ipywidgets.VBox([random_slid, normalize_slid])
hbox = ipywidgets.HBox([vbox, vbox2], layout=layout)


def update(mu, epochs, random=0, minorm=0):
    
    y = np.zeros(len(y_ref))
    if random:
        w = np.random.randn(2)/3
    else:
        w = np.zeros(2)
    X = np.zeros(2)
    X[0] = 1
    y = np.zeros(len(y_ref))
    e = np.zeros(len(y_ref))

    w_all = np.zeros((len(y_ref), 2))
    y[0]=y_ref[0]
    
    for ep in range(epochs):
        for i in range(1, len(y_ref)):
            X[1] = y_ref[i - 1]
            y[i] = np.dot(w, X)
            e[i]= y_ref[i] - y[i]
            if minorm:
                mu = mu_orig/(1 + np.dot(X, X))
            dw = mu * e[i] * X
            w = w + dw
            if ep == epochs - 1:
                w_all[i] = w 
 

    ### Update plot ###
    y_pl.set_ydata(y)
    #e_pl.set_ydata(e)

    fig.canvas.draw_idle()
    
    ax2.clear()
    e_pl, = ax2.plot(t, e)
    
    ax3.clear()
    w_all_pl = ax3.plot(t, w_all)
    plt.ylabel("Hodnoty vah")


    print('\n\nAbsolutní chyba za poslední epochu',sum(abs(e)))
    print(f'\nIdentifikované parametry: c = {w[0]}   a = {w[1]}')

    display(Math(r'y(k)=a y(k-1) + c'))
    display(Math(fr'y(k)={w[1]} \cdot y(k-1) + {w[0]}'))
    
interactive_plot = ipywidgets.interactive_output(update, {"mu": mu_slid, "epochs": epochs_slid,
                                                          "random": random_slid, "minorm": normalize_slid})

display(hbox, interactive_plot)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [1]:


def ahooj(a = 7):
    v = ahoj
    ahoj = 
    return 3
    
    

ahoj = 5

ahooj(3)

3

Nyní máme vstup i výstup a můžeme tedy identifikovat systém.

In [2]:

y_order = 2
u_order = 1

x_len = 1 + y_order + u_order